In [27]:
using JuMP
using Gurobi

global arcs = [1 2; 1 3; 1 4; 2 3; 2 5; 3 5; 4 5]
# global paths = [[1 2 3 5],
#                 [1 2 5],
#                 [1 3 5],
#                 [1 4 5]] 

# global D = []
global N = collect(1:maximum(arcs))
origin = 1
destination = 5
global arcNum = length(arcs[:,1])

#The SP#
function LPMin(x, origin, destination,p)
    global arcs
    global N
    global arcNum

    gurobi_env = Gurobi.Env()
    m = Model(() -> Gurobi.Optimizer(gurobi_env))
    set_optimizer_attribute(m, "OutputFlag", 0)
#     setparams!(gurobi_env, Heuristics=0.0, Cuts = 0, OutputFlag = 0)
#     m = Model(() -> Gurobi.Optimizer(gurobi_env))
    
    #Flow variables
    @variable(m, y[1:arcNum]>=0)
    #Max capacity variable
    @variable(m, u)
    
    flow_cons = Array{JuMP.ConstraintRef}(undef, length(N)-1)
    ucap_cons = Array{JuMP.ConstraintRef}(undef, arcNum)
    xcap_cons = Array{JuMP.ConstraintRef}(undef, arcNum)

    #Flow constraints
    for i in N
        #Outgoing flows from source node add to 1
        if i == origin
            outgoing = findall(arcs[:,1].== origin)
            flow_cons[i] = @constraint(m, sum(y[j] for j in outgoing) == 1)
        end
        #Outflow - Inflow = 0 for all nodes that are neither source nor sink
        if i != origin && i != destination  
            outgoing = findall(arcs[:,1].== i)
            incoming = findall(arcs[:,2].== i)
            flow_cons[i] = @constraint(m, sum(y[j] for j in outgoing)-sum(y[j] for j in incoming) == 0)
        end
    end

    #Constraints y_k <= (1/p_k) * v
    #If p_k = 0, then y_k <= infty * v
    #y_k is then not dependent on v  
    #v can be as small as possible for any value of y_k in this case
    #So only constraints for non-zero p_k are added
    
    #At the moment, all p_k > 0
    #Size of ucap_cons needs to be adjusted if we allow some p_k to be 0
    p = ones(arcNum)
    for i = 1:arcNum
        if p[i] != 0
            ucap_cons[i] = @constraint(m, u >= y[i])
        end
    end
    
    #Constraints y_k <= 1-x_k
    #x here is an input and not a variable
    for i = 1:arcNum 
        xcap_cons[i] = @constraint(m, 1-x[i]>= y[i])  
    end

    @objective(m, Min, u)
    optimize!(m)
    
    
    current_Obj = JuMP.objective_value.(m)
    current_y = JuMP.value.(y)
    current_u = JuMP.value.(u)
    flow_dual = zeros(length(N)-1)
    u_dual = zeros(arcNum)
    xcap_dual = zeros(arcNum)
    
    
    for i = 1:arcNum
        xcap_dual[i] = JuMP.dual(xcap_cons[i]) #Duals corresponding to y_k <= 1-x_k
        u_dual[i] = JuMP.dual(ucap_cons[i]) #Duals corresponding to y_k <= v/p_k
    end
    
    for i = 1:length(N)-1
        flow_dual[i] = JuMP.dual(flow_cons[i])
    end
    
    println("SP Obj = ",current_Obj)
    println("y_sol = ", current_y)
    println("flow_dual", flow_dual)
    println("u_dual", u_dual)
    println("xcap_dual", xcap_dual)

    return current_Obj, flow_dual, u_dual, xcap_dual
end

x_now = zeros(arcNum)#[0,0,0]
x_now[4] = 1
x_now[1] = 1
obj, flow_dual, u_dual, xcap_dual = LPMin(x_now, origin, destination, ones(arcNum))


--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only
SP Obj = 0.5
y_sol = [0.0, 0.5, 0.5, 0.0, 0.0, 0.5, 0.5]
flow_dual[0.5, 0.0, 0.5, 0.5]
u_dual[0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.5]
xcap_dual[0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


(0.5, [0.5, 0.0, 0.5, 0.5], [0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.5], [0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])

In [ ]:
#Generate scenarios:
numScen = 3
p = Array{Float64}(undef, numScen, arcNum)
# for i = 1:numScen
#     rowTotal = 0
#     for j = 1:arcNum
#         r = rand(1:5)
#         p[i,j] = r
# #         rowTotal = rowTotal + r
#     end
#     p[i,:] = 1 ./ p[i,:] #./rowTotal
# end

#An instance of p:
p = [1.0 0.25 0.333333 0.25 0.2 1.0 0.5; 
    0.5 0.25 0.333333 0.5 0.5 1.0 0.5; 
    0.5 1.0 0.5 0.5 1.0 1.0 0.2]


# p = [1/64 1/32 1/4 1/2 1/8 1/16 1/64;
#     0 1/5 1/5 1/5 0 1/5 1/5;
#     1/4 0 1/5 0 1/5 1/10 1/4]

#Define MP:
global c = [5,14,15,6,12,15,7] #interdiction cost
global b = 10 #budget
gurobi_env = Gurobi.Env()
setparams!(gurobi_env, Heuristics=0.0, Cuts = 0, OutputFlag = 0)
MP = Model(() -> Gurobi.Optimizer(gurobi_env))

@variable(MP, x[1:arcNum], Bin)
@variable(MP, θ[1:numScen] <= 1e6)

#Budget Constraints
@constraint(MP, sum(c[i]*x[i] for i=1:arcNum) <= 8)

@objective(MP, Max, (1/numScen)*sum(θ[i] for i = 1:numScen))

last_Obj = -20
MP_Obj = -10
iter = 1

#At the moment: Make sure to have at least 2 iterations in to have 
#decent values for last_Obj and MP_Obj
while MP_Obj - last_Obj > 1 || iter <= 2
    
    last_Obj = MP_Obj
    optimize!(MP)
    println("\n\nIter ", iter)

    MP_Obj = JuMP.objective_value.(MP)
    x_now = JuMP.value.(x)
    println("Obj = ",MP_Obj)
    println("x = ", x_now,"\n\n")
    
    #Solve SP for each scenario
    for k = 1:numScen
        println("\nk = ", k)
        
        obj, flow_dual, u_dual, xcap_dual = LPMin(x_now, origin, destination, p[k,:])
        cons = @constraint(MP, θ[k] >= flow_dual[1]*1 + sum(xcap_dual[i]*(1-x[i]) for i = 1:arcNum)) 
        println(cons)
    end
    
    println("last_Obj = ", last_Obj)
    println("MP_Obj = ", MP_Obj)
    iter = iter+1 
end

In [ ]:
#Single MIP:
global c = [5,14,15,6,12,15,7] #interdiction cost
global b = 10 #budget
p = [1.0 0.25 0.333333 0.25 0.2 1.0 0.5]
gurobi_env = Gurobi.Env()
setparams!(gurobi_env, Heuristics=0.0, Cuts = 0, OutputFlag = 0)
MIP = Model(() -> Gurobi.Optimizer(gurobi_env))

@variable(MIP, x[1:arcNum], Bin)
@variable(MIP, y[1:arcNum]>=0)
@variable(MIP, u)

#Budget Constraints
@constraint(MIP, sum(c[i]*x[i] for i=1:arcNum) <= 100)
#Flow constraints
for i in N
    #Outgoing flows from source node add to 1
    if i == origin
        outgoing = findall(arcs[:,1].== origin)
        @constraint(MIP, sum(y[j] for j in outgoing) == 1)
    end
    #Outflow - Inflow = 0 for all nodes that are neither source nor sink
    if i != origin && i != destination  
        outgoing = findall(arcs[:,1].== i)
        incoming = findall(arcs[:,2].== i)
        @constraint(MIP, sum(y[j] for j in outgoing)-sum(y[j] for j in incoming) == 0)
    end
end

for i = 1:arcNum
    if p[i] != 0
        @constraint(MIP, u*1/p[i] >= y[i])
    end
end

#Constraints y_k <= 1-x_k
for i = 1:arcNum 
    @constraint(MIP, 1-x[i]>= y[i])  
end

@objective(MIP, Min, u)
println(MIP)
optimize!(MIP)
MP_Obj = JuMP.objective_value.(MIP)
x_now = JuMP.value.(x)
u_now = JuMP.value.(u)
y_now = JuMP.value.(y)
println("Obj = ",MP_Obj)
println("x = ", x_now)
println("u = ", u_now)
println("y = ", y_now)

In [45]:
#Single Non-linear:
using JuMP
using Gurobi

global arcs = [1 2; 1 3; 1 4; 2 3; 2 5; 3 5; 4 5]
global paths = [[1 2 3 5],
                [1 2 5],
                [1 3 5],
                [1 4 5]]
global numPaths = length(paths)
global N = collect(1:maximum(arcs))
origin = 1
destination = 5
global arcNum = length(arcs[:,1])
global c = [5,14,15,6,12,15,7] #interdiction cost
global b = 10 #budget
global nodeNum = maximum(arcs[:,2])
println("nodeNum = ", nodeNum)
const gurobi_env = Gurobi.Env()
global D = [1.0 1.0 0.0 0.0; 0.0 0.0 1.0 0.0; 0.0 0.0 0.0 1.0; 1.0 0.0 0.0 0.0; 0.0 1.0 0.0 0.0; 1.0 0.0 1.0 0.0; 0.0 0.0 0.0 1.0]
# x_now = ones(arcNum)


# setparams!(gurobi_env, OutputFlag = 0) #Heuristics=0.0, Cuts = 0, 
MIP = Model(() -> Gurobi.Optimizer(gurobi_env))

set_optimizer_attribute(MIP, "OutputFlag", 0)
# @variable(MIP, x[1:arcNum], Bin)
@variable(MIP, β[1:arcNum]>=0)
@variable(MIP, w)
@variable(MIP, γ[1:arcNum]>=0)

x =zeros(arcNum)
# x[1] = 1
x[4] = 1
#Budget Constraints
# @constraint(MIP, sum(c[i]*x[i] for i=1:arcNum) <= 4)
#Flow constraints
# @constraint(MIP, sum(u[k] for k =1:arcNum) <= 1)
for l = 1:numPaths
#     i = arcs[k,1]
#     j = arcs[k,2]
    @constraint(MIP, w - sum(D[k,l]*(γ[k] + β[k]) for k = 1:arcNum) <= 0)
    @constraint(MIP,  sum(γ[k] for k = 1:arcNum) <= 2)
end

@objective(MIP, Max, w + sum(β[k]*(x[k] - 1) for k = 1:arcNum)  )
println(MIP)
optimize!(MIP)
println("Status = ", termination_status(MIP))
MP_Obj = JuMP.objective_value.(MIP)
# x_now = JuMP.value.(x)
w_now = JuMP.value.(w)
β_now = JuMP.value.(β)
γ_now = JuMP.value.(γ)
println("Obj = ",MP_Obj)
# println("x = ", x_now)
println("w = ", w_now)
println("β = ", β_now)
println("γ = ", γ_now)

nodeNum = 5

--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only
Max w - β[1] - β[2] - β[3] - β[5] - β[6] - β[7]
Subject to
 -β[1] - β[4] - β[6] + w - γ[1] - γ[4] - γ[6] <= 0.0
 γ[1] + γ[2] + γ[3] + γ[4] + γ[5] + γ[6] + γ[7] <= 2.0
 -β[1] - β[5] + w - γ[1] - γ[5] <= 0.0
 γ[1] + γ[2] + γ[3] + γ[4] + γ[5] + γ[6] + γ[7] <= 2.0
 -β[2] - β[6] + w - γ[2] - γ[6] <= 0.0
 γ[1] + γ[2] + γ[3] + γ[4] + γ[5] + γ[6] + γ[7] <= 2.0
 -β[3] - β[7] + w - γ[3] - γ[7] <= 0.0
 γ[1] + γ[2] + γ[3] + γ[4] + γ[5] + γ[6] + γ[7] <= 2.0
 β[1] >= 0.0
 β[2] >= 0.0
 β[3] >= 0.0
 β[4] >= 0.0
 β[5] >= 0.0
 β[6] >= 0.0
 β[7] >= 0.0
 γ[1] >= 0.0
 γ[2] >= 0.0
 γ[3] >= 0.0
 γ[4] >= 0.0
 γ[5] >= 0.0
 γ[6] >= 0.0
 γ[7] >= 0.0

Status = OPTIMAL
Obj = 0.6666666666666667
w = 0.6666666666666667
β = [0.0, 0.0, 0.0, 0.666667, 0.0, 0.0, 0.0]
γ = [0.0, 0.666667, 0.0, 0.0, 0.666667, 0.0, 0.666667]


In [44]:
#Single Non-linear:
using JuMP
using Gurobi

global arcs = [1 2; 1 3; 1 4; 2 3; 2 5; 3 5; 4 5]
global paths = [[1 2 3 5],
                [1 2 5],
                [1 3 5],
                [1 4 5]]
global numPaths = length(paths)
global N = collect(1:maximum(arcs))
origin = 1
destination = 5
global arcNum = length(arcs[:,1])
global c = [5,14,15,6,12,15,7] #interdiction cost
global b = 10 #budget
global nodeNum = maximum(arcs[:,2])
println("nodeNum = ", nodeNum)
const gurobi_env = Gurobi.Env()
global D = [1.0 1.0 0.0 0.0; 0.0 0.0 1.0 0.0; 0.0 0.0 0.0 1.0; 1.0 0.0 0.0 0.0; 0.0 1.0 0.0 0.0; 1.0 0.0 1.0 0.0; 0.0 0.0 0.0 1.0]
# x_now = ones(arcNum)


# setparams!(gurobi_env, OutputFlag = 0) #Heuristics=0.0, Cuts = 0, 
MIP = Model(() -> Gurobi.Optimizer(gurobi_env))

set_optimizer_attribute(MIP, "OutputFlag", 0)
# @variable(MIP, x[1:arcNum], Bin)
# @variable(MIP, β[1:arcNum]>=0)
@variable(MIP, u)
@variable(MIP, z[1:numPaths]>=0)

x =zeros(arcNum)
# x[1] = 1
x[4] = 1
#Budget Constraints
# @constraint(MIP, sum(c[i]*x[i] for i=1:arcNum) <= 4)
#Flow constraints
@constraint(MIP, sum(z[l] for l=1:numPaths) == 2)

for k = 1:arcNum
#     i = arcs[k,1]
#     j = arcs[k,2]
    @constraint(MIP, u - sum(D[k,l]*(z[l]) for l = 1:numPaths) >= 0)
    @constraint(MIP,  -sum(D[k,l]*z[l] for l = 1:numPaths) >= x[k] - 1)
end

@objective(MIP, Min, u )
println(MIP)
optimize!(MIP)
println("Status = ", termination_status(MIP))
MP_Obj = JuMP.objective_value.(MIP)
# x_now = JuMP.value.(x)
z_now = JuMP.value.(z)
u_now = JuMP.value.(u)
# γ_now = JuMP.value.(γ)
println("Obj = ",MP_Obj)
# println("x = ", x_now)
println("z = ", z_now)
println("u = ", u_now)
# println("γ = ", γ_now)

nodeNum = 5

--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only
Min u
Subject to
 z[1] + z[2] + z[3] + z[4] == 2.0
 u - z[1] - z[2] >= 0.0
 -z[1] - z[2] >= -1.0
 u - z[3] >= 0.0
 -z[3] >= -1.0
 u - z[4] >= 0.0
 -z[4] >= -1.0
 u - z[1] >= 0.0
 -z[1] >= 0.0
 u - z[2] >= 0.0
 -z[2] >= -1.0
 u - z[1] - z[3] >= 0.0
 -z[1] - z[3] >= -1.0
 u - z[4] >= 0.0
 -z[4] >= -1.0
 z[1] >= 0.0
 z[2] >= 0.0
 z[3] >= 0.0
 z[4] >= 0.0

Status = OPTIMAL
Obj = 0.6666666666666667
z = [0.0, 0.666667, 0.666667, 0.666667]
u = 0.6666666666666667
